<a href="https://colab.research.google.com/github/DANU011/PyDataAnalytics/blob/main/%EB%94%B0%EB%A6%89%EC%9D%B4_%EB%8C%80%EC%97%AC%EB%9F%89_%EC%98%88%EC%B8%A1/notebook/%EB%94%B0%EB%A6%89%EC%9D%B4_%EB%8C%80%EC%97%AC%EB%9F%89_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 따릉이 대여량 예측

## 데이터 확인하기


In [1]:
import pandas as pd

bicycle = pd.read_csv('train.csv')

display(bicycle.head())
display(bicycle.info())
display(f"{bicycle['date_time'].iloc[0]} ~ {bicycle['date_time'].iloc[-1]}")

,date_time,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,number_of_rentals
0,2018-04-01,207.500,4.000,0.000,3.050,75.000,12.600,21.000,30.000,22994
1,2018-04-02,208.317,2.950,0.000,3.278,69.833,12.812,19.000,19.500,28139
2,2018-04-03,213.516,2.911,0.000,2.690,74.879,10.312,15.316,19.113,26817
3,2018-04-04,143.836,3.692,0.425,3.138,71.849,8.312,12.368,43.493,26034
4,2018-04-05,95.905,4.000,0.723,3.186,73.784,5.875,10.421,63.378,2833


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273 entries, 0 to 272
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   date_time                  273 non-null    object 
 1   wind_direction             273 non-null    float64
 2   sky_condition              273 non-null    float64
 3   precipitation_form         273 non-null    float64
 4   wind_speed                 273 non-null    float64
 5   humidity                   273 non-null    float64
 6   low_temp                   273 non-null    float64
 7   high_temp                  273 non-null    float64
 8   Precipitation_Probability  273 non-null    float64
 9   number_of_rentals          273 non-null    int64  
dtypes: float64(8), int64(1), object(1)
memory usage: 21.5+ KB


None

'2018-04-01 ~ 2020-06-30'

## 날짜 관련 피처 생성하기
bicycle.info() 정보를 보면 'date_time' 컬럼이 'object'형으로 문자열(string) 타입임.
모델에 수치형 변수를 넣고 학습시키기 위해선 문자열 타입을 숫자로 바꿔주는 작업이 필요.

bicycle의 'date_time' 컬럼 정보를 각각 년도('year'), 월('month'), 일('day)로 나누어 숫자형으로 변환. 다만 'year', 'month'는 첫행의 값이 1부터 시작하도록 처리.

In [2]:
def seperate_datetime(dataframe):
    year = []
    month = []
    day = []

    for date in dataframe.date_time:
        year_point, month_point, day_point = date.split('-') # - 기준으로 string을 나누고 list로 만듦 ex) '2016-04-01' -> ['2016', '04', '01']
        year.append(int(year_point)-2017)
        month.append(int(month_point)-3)
        day.append(int(day_point))
    return year, month, day

year, month, day = seperate_datetime(bicycle)

bicycle['year'] = year
bicycle['month'] = month
bicycle['day'] = day

bicycle.head()

,date_time,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,number_of_rentals,year,month,day
0,2018-04-01,207.500,4.000,0.000,3.050,75.000,12.600,21.000,30.000,22994,1,1,1
1,2018-04-02,208.317,2.950,0.000,3.278,69.833,12.812,19.000,19.500,28139,1,1,2
2,2018-04-03,213.516,2.911,0.000,2.690,74.879,10.312,15.316,19.113,26817,1,1,3
3,2018-04-04,143.836,3.692,0.425,3.138,71.849,8.312,12.368,43.493,26034,1,1,4
4,2018-04-05,95.905,4.000,0.723,3.186,73.784,5.875,10.421,63.378,2833,1,1,5


## 피처 데이터와 정답지(target) 데이터로 나누기

우리의 목적은 주어진 날씨 데이터(low_temp, high_temp, precipitaion, wind_speed, humidity, insolation, cloud, year, month, day)를 이용하여 사람들이 따릉이를 얼마나 빌리는지(number_of_rentals) 예측하는 것.

날씨 데이터가 X, 따릉이 대여 수가 y.

이미 year, month, day로 컬럼이 생성되어 더 이상 불필요한 'date_time' 컬럼과 타겟변수 'number_of_rentals'를 제거하여 피처 데이터셋 X을 만들고 정답지(taget
) 데이터 y로 나누기.



In [3]:
X  = bicycle.drop(['date_time', 'number_of_rentals'], axis = 1)
y = bicycle.number_of_rentals
X

,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,year,month,day
0,207.500,4.000,0.000,3.050,75.000,12.600,21.000,30.000,1,1,1
1,208.317,2.950,0.000,3.278,69.833,12.812,19.000,19.500,1,1,2
2,213.516,2.911,0.000,2.690,74.879,10.312,15.316,19.113,1,1,3
3,143.836,3.692,0.425,3.138,71.849,8.312,12.368,43.493,1,1,4
4,95.905,4.000,0.723,3.186,73.784,5.875,10.421,63.378,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...
268,228.662,3.980,0.223,2.271,78.378,20.500,27.526,36.486,3,3,26
269,207.770,2.865,0.081,1.794,78.412,20.812,28.842,21.081,3,3,27
270,282.568,1.730,0.000,1.820,72.736,21.000,29.053,7.297,3,3,28
271,137.027,2.257,0.088,2.043,70.473,19.625,26.000,15.541,3,3,29


## 평가 산식과 기본 모델 fitting 성능 확인

처리를 하지 않은 original 모델의 성능을 확인.  
LinearRegression 모델을 사용하여, 위에서 만든 데이터셋을 이용하여 모델을 학습시키고, 예측하여 NMAE을 이용하여 모델 fitting 성능을 도출.

In [4]:
from sklearn.linear_model import LinearRegression

# metric 정의
import numpy as np

def NMAE(true, pred):
    score = np.mean(np.abs(true-pred) / true)
    return score

model = LinearRegression() # 모델 정의
model.fit(X, y) # 학습

y_hat = model.predict(X) # y 예측

score = NMAE(y, y_hat)

print(f'모델 NMAE: {score}')
# 이 점수는 학습데이터 셋. 모델이 학습 데이터에 대해 얼마나 잘 적합했는지를 나타냄.
# 실제 모델의 일반화 성능을 평가하려면 검증 데이터셋이나 테스트 데이터셋에 대하여 NMAE를 계산해야함.

모델 NMAE: 0.3234797521201454


## Feature Engineering by Human 1 : 요일 정보 추가
Feature Engineering :  데이터를 분석하기 위해 필요하다고 생각하는 정보를 만들어서 추가해주는 작업.

주어진 데이터로 부터 새로운 피처를 생성하는 목적:  
 - 모델이 데이터에서 더 많은 패턴을 학습할 수 있기 위하여
 - 데이터의 표현력을 증대시키기 위하여
 - 원래 데이터에 없던 새로운 정보를 추출하기 위하여
 - 데이터의 잠재적인 정보를 드러내기 위하여


먼저 직접 보면서 필요한 정보가 무엇일지 추론하고, 그 정보를 추가.

요일 정보 추가 :  
따릉이를 이용하는 사람들 중에는 휴일에 놀러가기 위해 이용하는 사람들도 있을 것이다. 이를 고려하면 따릉이 이용자 수를 예측하는 데에 요일 정보가 도움이 될 것이다.


In [5]:
# pd.to_datetime 함수를 사용하여 'date_time' 열의 문자열을 datetime 객체로 변환.
# dt.day_name() 메서드를 호출하여 각 날짜에 해당하는 요일의 이름을 문자열로 가져온다.
# LabelEncoder를 생성하고 fit 메서드로 요일 이름 데이터에 포함된 고유한 값들을 학습.
# transform 메서드를 사용하여 각 요일 이름을 해당하는 정수 값으로 변환.
from sklearn.preprocessing import LabelEncoder

# 사람 손으로 만든 feature들을 사용.
X_human = X.copy()

# 요일 정보 추가
week_day = pd.to_datetime(bicycle['date_time']).dt.day_name()

le = LabelEncoder()
le.fit(week_day)
X_human['week_day'] = le.transform(week_day)
X_human.head()

,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,year,month,day,week_day
0,207.500,4.000,0.000,3.050,75.000,12.600,21.000,30.000,1,1,1,3
1,208.317,2.950,0.000,3.278,69.833,12.812,19.000,19.500,1,1,2,1
2,213.516,2.911,0.000,2.690,74.879,10.312,15.316,19.113,1,1,3,5
3,143.836,3.692,0.425,3.138,71.849,8.312,12.368,43.493,1,1,4,6
4,95.905,4.000,0.723,3.186,73.784,5.875,10.421,63.378,1,1,5,4


## Feature Engineering by Human 2 : 온도 정보 추가
밖이 너무 춥거나, 너무 덥다면 밖에 나가고 싶지 않을 것이다.  
따릉이 이용자도 마찬가지로 날씨가 따릉이를 타기에 적절하지 않다면 대중교통이나 택시를 이용할 것이다.  
이 인사이트를 바탕으로 아래 3가지 날씨에는 따릉이 이용자 수가 적을 것이라는 가정을 세운다.
 - 일교차가 큰 날씨
 - 덥고 습한 날씨
 - 춥고 바람부는 날씨  


In [7]:
# 1. "일교차가 너무 큰 날씨"를 알기 위한 정보
X_human['temp_diff_info'] = X_human['high_temp'] - X_human['low_temp']

# 2. "덥고 습한 날씨"를 알기 위한 정보
X_human['sweat_info'] = X_human['high_temp'] * X_human['humidity']

# 3. "춥고 바람부는 날씨"를 알기 위한 정보
X_human['cold_info'] = X_human['low_temp'] * X_human['wind_speed']

X_human.head()

,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,year,month,day,week_day,temp_diff_info,sweat_info,cold_info
0,207.500,4.000,0.000,3.050,75.000,12.600,21.000,30.000,1,1,1,3,8.400,1575.000000,38.430000
1,208.317,2.950,0.000,3.278,69.833,12.812,19.000,19.500,1,1,2,1,6.188,1326.827000,41.997736
2,213.516,2.911,0.000,2.690,74.879,10.312,15.316,19.113,1,1,3,5,5.004,1146.846764,27.739280
3,143.836,3.692,0.425,3.138,71.849,8.312,12.368,43.493,1,1,4,6,4.056,888.628432,26.083056
4,95.905,4.000,0.723,3.186,73.784,5.875,10.421,63.378,1,1,5,4,4.546,768.903064,18.717750


## 새로운 feature추가에 따른 성능 변화 확인

In [9]:
# Feature Engineering by Human 모델 성능 확인

model = LinearRegression() # 모델 정의
model.fit(X_human, y) # 학습

y_hat = model.predict(X_human) # y 예측

score = NMAE(y, y_hat)

print(f'모델 NMAE: {score}')

모델 NMAE: 0.28426261278332077


## Feature Engineering by Computer
컴퓨터의 단순 계산만을 이용해서 feature를 추가.  
sklearn 등에서 제공하는 자동 feature engineering이 이런 방식.  
단순히 두 변수간의 곱을 추가해준 다음, 성능이 어떻게 변하는지 확인.

In [11]:
# 컴퓨터로 만든 feature들을 사용합니다.
X_computer = X.copy()

col_list = X_computer.columns

# feature 자기 자신의 제곱과 두 feature간의 곱이라는 새로운 feature를 추가.
for i in range(len(col_list)):
    for j in range(i, len(col_list)):
        X_computer[f'{col_list[i]}*{col_list[j]}'] = X_computer[col_list[i]] * X_computer[col_list[j]]

X_computer.head()

,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,year,month,...,Precipitation_Probability*Precipitation_Probability,Precipitation_Probability*year,Precipitation_Probability*month,Precipitation_Probability*day,year*year,year*month,year*day,month*month,month*day,day*day
0,207.500,4.000,0.000,3.050,75.000,12.600,21.000,30.000,1,1,...,900.000000,30.000,30.000,30.000,1,1,1,1,1,1
1,208.317,2.950,0.000,3.278,69.833,12.812,19.000,19.500,1,1,...,380.250000,19.500,19.500,39.000,1,1,2,1,2,4
2,213.516,2.911,0.000,2.690,74.879,10.312,15.316,19.113,1,1,...,365.306769,19.113,19.113,57.339,1,1,3,1,3,9
3,143.836,3.692,0.425,3.138,71.849,8.312,12.368,43.493,1,1,...,1891.641049,43.493,43.493,173.972,1,1,4,1,4,16
4,95.905,4.000,0.723,3.186,73.784,5.875,10.421,63.378,1,1,...,4016.770884,63.378,63.378,316.890,1,1,5,1,5,25


In [12]:
# feature 77개로 증가

## 자동 연산 통해 생성된 feature의 성능 확인

In [13]:
# Feature Engineering by Computer 모델 성능 확인​
model = LinearRegression() # 모델 정의
model.fit(X_computer, y) # 학습

y_hat = model.predict(X_computer) # y 예측

score = NMAE(y, y_hat)

print(f'모델 NMAE: {score}')

모델 NMAE: 0.14013355728695293


## Feature Engineering by Human & Computer
사람이 추가한 정보와 컴퓨터가 추가한 정보를 혼용

In [16]:
from sklearn.preprocessing import LabelEncoder

# Feature Engineering by Human & Computer은 Feature Engineering by Computer에 사람의 도메인 지식을 추가한 형태.
X_combination = X.copy()

# 요일 정보 추가
week_day = pd.to_datetime(bicycle['date_time']).dt.day_name()
le = LabelEncoder()
le.fit(week_day)
X_combination['week_day'] = le.transform(week_day)

# 1. "일교차가 너무 큰 날씨"를 알기 위한 정보
X_combination['temp_diff_info'] = X_combination['high_temp'] - X_combination['low_temp']

# 2. "덥고 습한 날씨"를 알기 위한 정보
X_combination['sweat_info'] = X_combination['high_temp'] * X_combination['humidity']

# 3. "춥고 바람부는 날씨"를 알기 위한 정보
X_combination['cold_info'] = X_combination['low_temp'] * X_combination['wind_speed']

## 자동 연산 통한 피처 생성
변수 자기 자신의 제곱과 두 변수간의 곱이라는 새로운 변수를 추가

In [18]:
col_list = X_combination.columns

# 변수 자기 자신의 제곱과 두 변수간의 곱이라는 새로운 변수를 추가.
for i in range(len(col_list)):
    for j in range(i, len(col_list)):
        X_combination[f'{col_list[i]}*{col_list[j]}'] = X_combination[col_list[i]] * X_combination[col_list[j]]

X_combination

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
<ipython-input-18-45090e86d664>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_combination[f'{col_list[i]}*{col_list[j]}'] = X_combination[col_list[i]] * X_combination[col_list[j]]
<ipython-input-18-45090e86d664>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_combination[f'{col_list[i]}*{col_list[j]}'] = X_combination[col_list[i]] * X_combination[col_list[j]]
<ipython-input-18-45090e86d664>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `f

,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,year,month,...,temp_diff_info*cold_info*temp_diff_info*cold_info,temp_diff_info*cold_info*sweat_info*sweat_info,temp_diff_info*cold_info*sweat_info*cold_info,temp_diff_info*cold_info*cold_info*cold_info,sweat_info*sweat_info*sweat_info*sweat_info,sweat_info*sweat_info*sweat_info*cold_info,sweat_info*sweat_info*cold_info*cold_info,sweat_info*cold_info*sweat_info*cold_info,sweat_info*cold_info*cold_info*cold_info,cold_info*cold_info*cold_info*cold_info
0,207.500,4.000,0.000,3.050,75.000,12.600,21.000,30.000,1,1,...,104207.587344,8.007755e+08,1.953892e+07,4.767497e+05,6.153500e+12,1.501454e+11,3.663548e+09,3.663548e+09,8.939057e+07,2.181130e+06
1,208.317,2.950,0.000,3.278,69.833,12.812,19.000,19.500,1,1,...,67538.648918,4.575144e+08,1.448159e+07,4.583824e+05,3.099254e+12,9.809995e+10,3.105134e+09,3.105134e+09,9.828606e+07,3.111025e+06
2,213.516,2.911,0.000,2.690,74.879,10.312,15.316,19.113,1,1,...,19267.482391,1.825674e+08,4.415837e+06,1.068078e+05,1.729902e+12,4.184190e+10,1.012048e+09,1.012048e+09,2.447885e+07,5.920805e+05
3,143.836,3.692,0.425,3.138,71.849,8.312,12.368,43.493,1,1,...,11192.132430,8.354045e+07,2.452083e+06,7.197362e+04,6.235637e+11,1.830287e+10,5.372264e+08,5.372264e+08,1.576869e+07,4.628432e+05
4,95.905,4.000,0.723,3.186,73.784,5.875,10.421,63.378,1,1,...,7240.459816,5.030675e+07,1.224640e+06,2.981195e+04,3.495315e+11,8.508802e+09,2.071336e+08,2.071336e+08,5.042345e+06,1.227480e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,228.662,3.980,0.223,2.271,78.378,20.500,27.526,36.486,3,3,...,106993.718512,1.522487e+09,3.285394e+07,7.089590e+05,2.166452e+13,4.675013e+11,1.008827e+10,1.008827e+10,2.176959e+08,4.697686e+06
269,207.770,2.865,0.081,1.794,78.412,20.812,28.842,21.081,3,3,...,89888.390128,1.533443e+09,2.531605e+07,4.179500e+05,2.615963e+13,4.318769e+11,7.129980e+09,7.129980e+09,1.177109e+08,1.943323e+06
270,282.568,1.730,0.000,1.820,72.736,21.000,29.053,7.297,3,3,...,94732.012502,1.374451e+09,2.485876e+07,4.496036e+05,1.994167e+13,3.606715e+11,6.523222e+09,6.523222e+09,1.179811e+08,2.133844e+06
271,137.027,2.257,0.088,2.043,70.473,19.625,26.000,15.541,3,3,...,65330.569240,8.581248e+08,1.877727e+07,4.108793e+05,1.127157e+13,2.466416e+11,5.396949e+09,5.396949e+09,1.180946e+08,2.584117e+06


## 성능 확인

In [19]:
# Feature Engineering by Human & Computer  모델 성능 확인

model = LinearRegression() # 모델 정의
model.fit(X_combination, y) # 학습

y_hat = model.predict(X_combination) # y 예측

score = NMAE(y, y_hat)

print(f'모델 NMAE: {score}')

모델 NMAE: 1.9954394991999564e-10
